In [183]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

1. 데이터를 읽어 들임


In [184]:
# 데이터의 경로 지정

path_data_in = './Data_in/'

uriage = pd.read_csv(f'{path_data_in}uriage.csv') # 매출이력 (기간 == 2019 1월~ 2019 7월)

cs = pd.read_excel(f'{path_data_in}kokyaku_daicho.xlsx') # 대리점에서 관리하는 고객 정보

data_dump = pd.read_csv(f'{path_data_in}dump_data.csv')

In [185]:
cs.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   고객이름    200 non-null    object
 1   지역      200 non-null    object
 2   등록일     200 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


In [186]:
uriage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   purchase_date  2999 non-null   object 
 1   item_name      2999 non-null   object 
 2   item_price     2612 non-null   float64
 3   customer_name  2999 non-null   object 
dtypes: float64(1), object(3)
memory usage: 93.8+ KB


In [187]:
# bool(list(cs['고객이름']) in list(uriage['고객이름']))

In [188]:
# 데이터를 결합

uriage.columns = ['구매날짜', '상품이름', '상품가격', '고객이름']

# df = pd.merge(uriage, cs, how = 'left', on= '고객이름')

# df

2. 데이터의 오류를 살피기


In [189]:
# 입력 오류나, 표기 방법의 차이가 부정합을 일으킴
    # 데이터의 정합성에 문제가 있음

cs.isna().sum()
# uriage.isna().sum() # item_price에서 387개의 결측치 발견
# data_dump.isna().sum()


고객이름    0
지역      0
등록일     0
dtype: int64

In [190]:
uriage.head() # 상품 이름이 통일되지 않음 / 상품 가격, 지역, 등록일에 nan

,구매날짜,상품이름,상품가격,고객이름
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [191]:
uriage.sort_values('구매날짜', axis= 0, ignore_index= True, inplace= True)

uriage

,구매날짜,상품이름,상품가격,고객이름
0,2019-01-01 0:20,상품O,1500.0,김민석
1,2019-01-01 12:28,상품S,1900.0,김현준
2,2019-01-01 13:11,상품E,500.0,김은율
3,2019-01-01 15:40,상품Q,1700.0,김우찬
4,2019-01-01 15:47,상품J,1000.0,김동욱
...,...,...,...,...
2994,2019-07-31 22:11,상품r,NaN,김민혁
2995,2019-07-31 23:05,상품L,1200.0,김주혁
2996,2019-07-31 4:22,상품F,600.0,김승재
2997,2019-07-31 4:58,상품 V,2200.0,김윤호


3. 데이터에 오류가 있는 상태로 집계


In [192]:
uriage.info()
uriage.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구매날짜    2999 non-null   object 
 1   상품이름    2999 non-null   object 
 2   상품가격    2612 non-null   float64
 3   고객이름    2999 non-null   object 
dtypes: float64(1), object(3)
memory usage: 93.8+ KB


,구매날짜,상품이름,상품가격,고객이름
0,2019-01-01 0:20,상품O,1500.0,김민석
1,2019-01-01 12:28,상품S,1900.0,김현준
2,2019-01-01 13:11,상품E,500.0,김은율
3,2019-01-01 15:40,상품Q,1700.0,김우찬
4,2019-01-01 15:47,상품J,1000.0,김동욱


In [193]:
# '매출 이력'에서 상품별로 월 매출 합계를 집계
# index == purchase_month(년월만 추출), columns == item_name, size

# 년월만 추출

uriage['구매날짜'] = pd.to_datetime(uriage['구매날짜'], format='%Y-%m-%d %H:%M')

uriage['구매년월'] = uriage['구매날짜'].dt.strftime('%Y-%m')

uriage['구매년월']


0       2019-01
1       2019-01
2       2019-01
3       2019-01
4       2019-01
         ...   
2994    2019-07
2995    2019-07
2996    2019-07
2997    2019-07
2998    2019-07
Name: 구매년월, Length: 2999, dtype: object

In [194]:
# pivot_table 생성

uriage_na = uriage.pivot_table(
    index= '구매년월',
    columns = '상품이름',
    values= '상품가격',
    aggfunc= 'sum'
)

uriage_na

상품이름,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
구매년월,,,,,,,,,,,,,,,,,,,,,
2019-01,1400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1100.0,1200.0,1500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02,NaN,NaN,NaN,NaN,NaN,NaN,2400.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1900.0,2000.0,2200.0,NaN,NaN
2019-03,NaN,500.0,1300.0,1600.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2300.0,NaN,1700.0,...,NaN,NaN,NaN,NaN,NaN,1900.0,NaN,NaN,NaN,NaN
2019-05,NaN,NaN,NaN,NaN,1900.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2500.0
2019-06,NaN,NaN,NaN,NaN,NaN,2300.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1600.0,NaN,NaN,NaN,NaN,2400.0,NaN
2019-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,1500.0,NaN,1800.0,NaN,NaN,NaN,NaN,NaN


In [195]:
uriage_na = uriage.pivot_table(
    index= '구매년월',
    columns = '상품이름',
    aggfunc= 'size',
    fill_value= 0
)

uriage_na

상품이름,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
구매년월,,,,,,,,,,,,,,,,,,,,,
2019-01,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
2019-02,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
2019-03,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-04,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
2019-05,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2019-06,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2019-07,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [196]:
# 매출집계

uriage.pivot_table(
    index= '구매년월',
    columns = '상품이름',
    values= '상품가격',
    aggfunc= 'sum',
    fill_value= 0
)


상품이름,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
구매년월,,,,,,,,,,,,,,,,,,,,,
2019-01,1400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1100.0,1200.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02,0.0,0.0,0.0,0.0,0.0,0.0,2400.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,2000.0,2200.0,0.0,0.0
2019-03,0.0,500.0,1300.0,1600.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,1700.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0
2019-05,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0
2019-06,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1600.0,0.0,0.0,0.0,0.0,2400.0,0.0
2019-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1500.0,0.0,1800.0,0.0,0.0,0.0,0.0,0.0


4. 상품명의 오류를 수정


In [197]:
# 상품 == A~Z까지 총 26개여야 함
    # 현재 갯수 == 99개
        # 문제점
            # 상품명 뒤의 알파벳의 대/소문자 문제
            # 상품명에 들어가 있는 공백문제

In [198]:
# 모두 대문자로 변경

uriage['상품이름'] =\
    uriage['상품이름'].map(
    lambda x : x.upper()
)

In [199]:
# 공백 제거

uriage['상품이름'] =\
    uriage['상품이름'].map(
    lambda x : x.replace(' ', '')
)

uriage.sort_values(['상품이름'], ignore_index= True, inplace= True)

In [205]:
uriage

,구매날짜,상품이름,상품가격,고객이름,구매년월
0,2019-04-06 11:06:00,상품A,NaN,김태인,2019-04
1,2019-03-12 02:04:00,상품A,100.0,김서윤,2019-03
2,2019-06-03 17:33:00,상품A,100.0,김서진,2019-06
3,2019-01-28 22:13:00,상품A,NaN,김윤호,2019-01
4,2019-03-20 02:16:00,상품A,100.0,김라온,2019-03
...,...,...,...,...,...
2994,2019-06-13 07:18:00,상품Y,2500.0,김민규,2019-06
2995,2019-05-22 17:55:00,상품Y,2500.0,김한결,2019-05
2996,2019-03-03 13:01:00,상품Y,2500.0,김대현,2019-03
2997,2019-06-01 18:57:00,상품Y,2500.0,김서준,2019-06


In [200]:
# 결과 검증

uriage['상품이름'].describe()

count     2999
unique      26
top        상품S
freq       140
Name: 상품이름, dtype: object

5. 금액의 결측치를 수정


In [201]:
# 결측치 확인

uriage.isna().any()

uriage.isna().sum()

# uriage['상품가격'].describe()

구매날짜      0
상품이름      0
상품가격    387
고객이름      0
구매년월      0
dtype: int64

In [202]:
# 특이사항 == 집계 기간에 상품 단가의 변동이 없다는 전제조건이 있음

In [178]:
#  결측치의 조건을 flag로 설정

flag = uriage['상품가격'].isna()


In [204]:
flag

0        True
1       False
2        True
3       False
4        True
        ...  
2994    False
2995    False
2996    False
2997    False
2998    False
Name: 상품가격, Length: 2999, dtype: bool

In [208]:
for idx, trg in enumerate(list(uriage.loc[flag, '상품이름'].unique())): # enumerate() == index와 value를 같이 뽑을 때 사용
    print(idx, trg) # 상품 이름중 '상품Z'가 출력되지 않은 이유는 '상품Z'에는 결측치가 없기에 loc에서 flag 조건으로 뽑히지 않음

0 상품A
1 상품B
2 상품C
3 상품D
4 상품E
5 상품F
6 상품G
7 상품H
8 상품I
9 상품J
10 상품K
11 상품L
12 상품M
13 상품N
14 상품O
15 상품P
16 상품Q
17 상품R
18 상품S
19 상품T
20 상품U
21 상품V
22 상품W
23 상품X
24 상품Y


6. 고객 이름의 오류를 수정


In [167]:
uriage['고객이름']

1368    김태인
1038    김서윤
2168    김서진
377     김윤호
1147    김라온
       ... 
2307    김민규
1987    김한결
904     김대현
2142    김서준
611     김재현
Name: 고객이름, Length: 2999, dtype: object

7. 날짜 오류를 수정


8. 고객 이름을 키로 두 개의 데이터를 결합(조인)


9. 정제한 데이터를 덤프


10. 데이터를 집계